## Setup
1. Create EventHouse (source + destination)
2. Query data



## Work with data in a Microsoft Fabric eventhouse

1. Use 'project' and 'take' to view a sample number of records in the table and check the data.
 
            Bikestream
            | project Street, No_Bikes
            | take 10

2. Rename columns

            Bikestream 
            | project Street, ["Number of Empty Docks"] = No_Empty_Docks
            | take 10

3. Aggregate (group the number of bikes by neighbourhood to determine the amount of available bikes in each neighbourhood)

            Bikestream
            | summarize ["Total Number of Bikes"] = sum(No_Bikes) by Neighbourhood
            | project Neighbourhood, ["Total Number of Bikes"]

4. Trnasform data (group all trips for which the neighbourhood is unknown into a Unidentified category)

            Bikestream
            | summarize ["Total Number of Bikes"] = sum(No_Bikes) by Neighbourhood
            | project Neighbourhood = case(isempty(Neighbourhood) or isnull(Neighbourhood), "Unidentified", Neighbourhood), ["Total Number of Bikes"]

5. Sort data

            Bikestream
            | summarize ["Total Number of Bikes"] = sum(No_Bikes) by Neighbourhood
            | project Neighbourhood = case(isempty(Neighbourhood) or isnull(Neighbourhood), "Unidentified", Neighbourhood), ["Total Number of Bikes"]
            | sort by Neighbourhood asc

6. Filter (filter the bike data to include only bike points in the Chelsea neighbourhood)

            Bikestream
            | where Neighbourhood == "Chelsea"
            | summarize ["Total Number of Bikes"] = sum(No_Bikes) by Neighbourhood
            | project Neighbourhood = case(isempty(Neighbourhood) or isnull(Neighbourhood), "Unidentified", Neighbourhood), ["Total Number of Bikes"]
            | sort by Neighbourhood asc

## T-SQL endpoint 

1. Query data

 SELECT TOP 10 Street, No_Bikes
 FROM Bikestream

 SELECT TOP 10 Street, No_Empty_Docks as [Number of Empty Docks]
 from Bikestream

2. Summaize

total number of bikes available:
            
            SELECT sum(No_Bikes) AS [Total Number of Bikes]
            FROM Bikestream

group the total number of bikes by neighbourhood:

            SELECT Neighbourhood, Sum(No_Bikes) AS [Total Number of Bikes]
            FROM Bikestream
            GROUP BY Neighbourhood


CASE statement to group bike points with an unknown origin into a Unidentified category for follow-up.

            SELECT CASE
                    WHEN Neighbourhood IS NULL OR Neighbourhood = '' THEN 'Unidentified'
                    ELSE Neighbourhood
                    END AS Neighbourhood,
                    SUM(No_Bikes) AS [Total Number of Bikes]
            FROM Bikestream
            GROUP BY CASE
                        WHEN Neighbourhood IS NULL OR Neighbourhood = '' THEN 'Unidentified'
                        ELSE Neighbourhood
                    END;

3. Sort (order the grouped results by neighbourhood)

            SELECT CASE
                    WHEN Neighbourhood IS NULL OR Neighbourhood = '' THEN 'Unidentified'
                    ELSE Neighbourhood
                END AS Neighbourhood,
                SUM(No_Bikes) AS [Total Number of Bikes]
            FROM Bikestream
            GROUP BY CASE
                    WHEN Neighbourhood IS NULL OR Neighbourhood = '' THEN 'Unidentified'
                    ELSE Neighbourhood
                    END
            ORDER BY Neighbourhood ASC;

4. Filter (filter the grouped data so that only rows having a neighbourhood of “Chelsea” are included in the results)

            SELECT CASE
                    WHEN Neighbourhood IS NULL OR Neighbourhood = '' THEN 'Unidentified'
                    ELSE Neighbourhood
                    END AS Neighbourhood,
                    SUM(No_Bikes) AS [Total Number of Bikes]
            FROM Bikestream
            GROUP BY CASE
                        WHEN Neighbourhood IS NULL OR Neighbourhood = '' THEN 'Unidentified'
                        ELSE Neighbourhood
                    END
            HAVING Neighbourhood = 'Chelsea'
            ORDER BY Neighbourhood ASC;
